In [1]:
# uncomment the following if you had not installed the package
# import sys
# sys.path.append('/home/khan/Documents/developments/pyIntertidalDEM')

from pyintdem.data import Database, DataFile
from pyintdem.util import create_mask
from pathlib import Path
import rioxarray
from tqdm import tqdm


# Example for downloaded dataset

In [14]:
datadir = Path('/run/media/khan/Workbench/Developments/Bengal/Data')
database = Database(datadir)

In [15]:
database

{'T46QBK': [{'filetype': 'theia',
   'fpath': '/run/media/khan/Workbench/Developments/Bengal/Data/T46QBK/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8.zip',
   'mission': 'SENTINEL2A',
   'product': 'L2A',
   'version': 'V1-8',
   'time': Timestamp('2016-01-18 04:43:55.092000'),
   'tile': 'T46QBK',
   'bands': {'B11': '/vsizip//run/media/khan/Workbench/Developments/Bengal/Data/T46QBK/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8.zip/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8_FRE_B11.tif',
    'B4': '/vsizip//run/media/khan/Workbench/Developments/Bengal/Data/T46QBK/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8.zip/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8_FRE_B4.tif',
    'B6': '/vsizip//run/media/khan/Workbench/Developments/Bengal/Data/T46QBK/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8.zip/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8/SENTINEL2A_20160118-0443

In [17]:
maskdir = Path('/run/media/khan/Workbench/Developments/Bengal/Masks')
create_mask(database=database, maskdir=maskdir, nmask=0.5, ext='tif', band='B11')

In [22]:
database.tiles

['T46QBK', 'T46QCK']

# Example for Pekel et al. dataset

In [2]:
rootdir = Path('F:/Sentinel2')
datadir = rootdir / 'Data'
database = Database(datadir)

C:\Users\khan\miniconda3\envs\pyintdem\Lib\site-packages\rasterio\__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [48]:
# Merging Pekel dataset
# Run once only
from rioxarray.merge import merge_arrays

pekel_dir = Path('F:/Sentinel2/Auxiliary/Pekel_etal_2016/')

fnames = [
    'occurrence_40E_10Sv1_4_2021.tif',
    'occurrence_40E_20Sv1_4_2021.tif',
    'occurrence_50E_10Sv1_4_2021.tif',
    'occurrence_50E_20Sv1_4_2021.tif'
]

datasets = [rio.open_rasterio(pekel_dir / fname) for fname in fnames]
merged_ds = merge_arrays(datasets)
merged_ds.rio.to_raster('F:/Sentinel2/Auxiliary/Pekel_etal_2016/occurrence_madagascar.tif')

In [57]:
ds_mask_global = rioxarray.open_rasterio('F:/Sentinel2/Auxiliary/Pekel_etal_2016/occurrence_madagascar.tif')

In [62]:
maskdir = rootdir / 'Masks_Pekel'
if not maskdir.exists():
    maskdir.mkdir()

for tile in tqdm(database):
    ds_band = rioxarray.open_rasterio(database[tile][0]['bands']['B11'])
    ds_mask_local = ds_mask_global.rio.reproject_match(ds_band)

    # Mask land 0, water 1
    ds_mask_local = (ds_mask_local >= 25).astype(int)
    ds_mask_local.rio.to_raster(maskdir / f'{tile}.tif')

100%|██████████| 60/60 [07:29<00:00,  7.49s/it]


In [11]:
database_test = {key: database[key] for key in  ['T39KTV', 'T38KMG', 'T39KTS']}
maskdir = rootdir / 'Masks_Sentinel2_Updated'
if not maskdir.exists():
    maskdir.mkdir()
create_mask(database=database_test, maskdir=maskdir, nmask=0.75, ext='tif', band='B11', normalize=True)

C:\Users\khan\Documents\Developments\pyIntertidalDEM\pyintdem\core.py:524: RuntimeWarning: invalid value encountered in divide
  data=self.data/other.data,
